This iteration of the API call program uses an artist list formatted as a csv as the input to do the API calls for album information.

In [43]:
import requests
import pandas as pd
import time

In [44]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': '9cdef0bc741e471385b100af843c28c5',
    'client_secret': 'a0820dd07a3e4d8facb51bb5ed5130a2',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [45]:
def get_albums(album_df):
    a = 0
    artist_id_list = csv

    while a < len(artist_id_list):
        if a < len(artist_id_list):
            #print(artist_id_list.iloc[a,0])
            artist_id = artist_id_list.iloc[a,0]
            # pull all artist's albums
            r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                            headers=headers, 
                            params={'include_groups': 'album'})
            d = r.json()
            
            time.sleep(1)
            
            for album in d['items']:
                if album['type'] == 'album':
                    band_id = album['artists'][0]['id']
                    band_name = album['artists'][0]['name']
                    album_name = album['name']
                    release_date = album['release_date']
                    total_tracks = album['total_tracks']
                    album_id = album['id']

                    # skip album doubles
                    trim_name = album_name.split('(')[0].strip()
                    # Trivium specific
                    if trim_name.upper() in albums or album['name'] == ('Ascendancy (Commentary Version)') or album['name'] == ('Ember To Inferno'):
                        continue
                    # append to album list for avoiding doubles
                    albums.append(trim_name.upper())
                    
                    #save data in pd dataframe
                    ##album_df = album_df.append({'band_id':band_id,'band_name':band_name,'album_name':album_name,'release_date':release_date,
                    ##                'total_tracks':total_tracks,'album_id':album_id},ignore_index=True)
                    temp_df = pd.DataFrame({'artist_id':band_id,'artist_name':band_name,'album_id':album_id,'album_name':album_name,'release_date':release_date,
                        'total_tracks':total_tracks}, index = [a])
                    album_df = pd.concat([album_df, temp_df], ignore_index=True)
        else:
            break 
        a += 1
    #return album_df
    return album_df

In [46]:
#Main
BASE_URL = 'https://api.spotify.com/v1/'
csv = pd.read_csv('artist_list.csv',index_col=False)
#data = []
albums = []
album_df = pd.DataFrame(columns=["artist_id","artist_name","album_id","album_name","release_date",
                  "total_tracks"])
album_df = get_albums(album_df)

#album_df= album_df.reset_index()
#album_df = album_df.set_index('album_id')
album_df.to_csv(r'C:\Users\dsgra\OneDrive\Documents\01 - Full Stack Data Science PJ\data science pj\album_list.csv',encoding='utf-8-sig')
print('-----------------------------------')
print('Saved to .csv in project directory.')
print('-----------------------------------')

-----------------------------------
Saved to .csv in project directory.
-----------------------------------


In [47]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_id     22 non-null     object
 1   artist_name   22 non-null     object
 2   album_id      22 non-null     object
 3   album_name    22 non-null     object
 4   release_date  22 non-null     object
 5   total_tracks  22 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB


In [54]:
import psycopg2
import numpy as np
import psycopg2.extras as extras
import pandas as pd
  
# bulk insert to postgreSQL db  
def execute_values(conn, album_df, table):
  
    tuples = [tuple(x) for x in album_df.to_numpy()]
  
    cols = ','.join(list(album_df.columns))
    # SQL query to execute
    query = """
        INSERT INTO %s(%s)
        VALUES %%s
        ON CONFLICT DO NOTHING
        """ % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
       # print(tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
conn = psycopg2.connect(database="dspjmain",
                        host="localhost",
                        user="postgres",
                        password="derekgrammer",
                        port="5432")
  
execute_values(conn, album_df, 'dspjschema.artists')


[('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '0mrtkWYrUzTuFwyiiQPdQs', 'In The Court Of The Dragon', '2021-10-08', 10), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '0aXIJYbWk4u41iJmoJmp8y', 'What The Dead Men Say', '2020-04-24', 10), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '3qq44o8Qqvz5JQ7b5AevLT', 'The Sin and the Sentence', '2017-10-20', 11), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '7mjOpLKh8UXgyXB1J5LvbN', 'Ember To Inferno: Ab Initio', '2016-12-02', 25), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '5l2xUBNPkPSZB0KVrEXANI', 'Silence in the Snow (Special Edition)', '2015-10-02', 13), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '5qI1swLFBDtgo9Q7lS2GcT', 'Vengeance Falls', '2013-10-11', 10), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '38sCkVGoLKjqFcOrTKTEC4', 'In Waves', '2011-08-03', 13), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '0kIXzVzbFuUf5kxM8US67m', 'Shogun (Special Edition)', '2008-09-24', 14), ('278ZYwGhdK6QTzE3MFePnP', 'Trivium', '42MixogSAsobX5po649Y0j', 'The Crusade', '2006-01-01', 13), ('278ZYwGhdK6QTzE3MFePn